In [ ]:
import os
import json
import requests

In [ ]:
HOST = "http://127.0.0.1"
PORT = "3012"

API_ENDPOINT = f"{HOST}:{PORT}/api"

In [ ]:
gemeente = "barendrecht"
BASE_PATH = f"data/{gemeente}/vergaderingen"

In [ ]:
def get_transcript(path):
    with open(path, "r") as f:
        data = json.load(f)

    filtered = [{"text:": d["text"], "start:": d["start"]} for d in data["segments"]]

    return filtered

def get_agendapunten(path):
    with open(path, "r") as f:
        data = json.load(f)

    return [d["agendaPoint"] for d in data]

In [ ]:
def translate_transcript(transcript):
    translated = []
    for s in transcript:
        r = requests.post(f"{API_ENDPOINT}/translateDE", json={"text": s["text"]})
        if r.status_code != 200:
            raise Exception("Error translating")

        translated.append({"text":r.json()["text"], "start": s["start"]})

    return translated


def translate_agendapunten(agenda):
    translated = []
    for p in agenda:
        r = requests.post(f"{API_ENDPOINT}/translateDE", json={"text": p})
        if r.status_code != 200:
            raise Exception("Error translating")

        translated.append(r.json()["text"])

    return translated

In [ ]:
for year in os.listdir(BASE_PATH):
    if year.startswith("."):
        continue

    agenda_dir = f"{BASE_PATH}/{year}/agendas"
    transcript_dir = f"{BASE_PATH}/{year}/transcripts"
    if not os.path.isdir(agenda_dir):
        print(f"{agenda_dir} does not exist")
        continue
    if not os.path.isdir(transcript_dir):
        print(f"{transcript_dir} does not exist")
        continue

    for trans_file in os.listdir(transcript_dir):
        agenda_name = trans_file.replace(".mp4", "")
        if not os.path.isfile(f"{agenda_dir}/agenda_name"):
            print("Transcript does exist, but agenda does not")
            continue

        # translated_transcript = translate_transcript(
        #     get_transcript(f"{transcript_dir}/trans_file")
        # )
        translated_agendapunten = translate_agendapunten(
            get_agendapunten(f"{agenda_dir}/agenda_name")
        )
        print(translated_agendapunten)
        break
        # TODO: Run high context window llm to seperate agendapunten on time.